In [1]:
import pandas as pd

# Path to your CSV file
file_path = r"C:\Users\mukta\OneDrive\Desktop\Algo_project\data\raw\companies.csv"

# Read CSV into a DataFrame
df = pd.read_csv(file_path)

In [2]:
# Display all column names
print(df.columns.tolist())

['ticker', 'company name', 'short name', 'industry', 'description', 'website', 'logo', 'ceo', 'exchange', 'market cap', 'sector', 'tag 1', 'tag 2', 'tag 3']


In [3]:
# Keep only selected columns
cols_to_keep = ['ticker', 'company name', 'market cap','sector']
cleaned_df = df[cols_to_keep]

# Display the result
print(cleaned_df.head())

  ticker                     company name    market cap              sector
0      A        Agilent Technologies Inc.  2.421807e+10          Healthcare
1     AA                Alcoa Corporation  5.374967e+09     Basic Materials
2   AABA                      Altaba Inc.  4.122368e+10  Financial Services
3    AAC                AAC Holdings Inc.  6.372010e+07          Healthcare
4   AADR  AdvisorShares Dorsey Wright ADR  1.031612e+08                 NaN


In [5]:
# Check for null values in each column
print(cleaned_df.isnull().sum())

ticker            0
company name      0
market cap        1
sector          950
dtype: int64


In [8]:
# Drop rows where 'market cap' or 'sector' is null
cleaned_df = cleaned_df.dropna(subset=['market cap', 'sector'])

# Drop rows where 'market cap' equals 0
cleaned_df = cleaned_df[cleaned_df["market cap"] != 0]

# Check if nulls are gone
print(cleaned_df.isnull().sum())


ticker          0
company name    0
market cap      0
sector          0
dtype: int64


In [13]:
import warnings
import logging
import yfinance as yf

# Hide warnings and yfinance error logs
warnings.filterwarnings("ignore")
logging.getLogger("yfinance").setLevel(logging.CRITICAL)

tickers = cleaned_df["ticker"].dropna().astype(str).unique().tolist()

has_data = 0
no_data = 0
failed_tickers = []
total = len(tickers)

print(f"📊 Checking {total} tickers for available data on Yahoo Finance...\n")

for i, tkr in enumerate(tickers, start=1):
    try:
        data = yf.download(tkr, period="60d", interval="1d", progress=False)
        if data is not None and not data.empty:
            has_data += 1
        else:
            no_data += 1
            failed_tickers.append(tkr)
    except Exception:
        no_data += 1
        failed_tickers.append(tkr)

    # progress update every 200 tickers
    if i % 200 == 0 or i == total:
        print(f"✅ Processed {i}/{total} | With data: {has_data} | Missing: {no_data}")
        if failed_tickers:
            print(f"🚫 Failed tickers so far ({len(failed_tickers)}): {', '.join(failed_tickers[-10:])} ...\n")

print("\n📈 Final Summary:")
print(f"✅ Tickers with data available: {has_data}")
print(f"🚫 Tickers with no data (possibly delisted): {no_data}")
print(f"🧾 Total unique failed tickers: {len(failed_tickers)}")


📊 Checking 5402 tickers for available data on Yahoo Finance...

✅ Processed 200/5402 | With data: 120 | Missing: 80
🚫 Failed tickers so far (80): AKS, AKTS, ALAC, ALBO, ALDR, ALGR, ALIM, ALJJ, ALLK, ALNA ...

✅ Processed 400/5402 | With data: 252 | Missing: 148
🚫 Failed tickers so far (148): ARRS, ARTX, ARYA, ASCMA, ASFI, ASLN, ASNA, AST, ASV, AT ...

✅ Processed 600/5402 | With data: 383 | Missing: 217
🚫 Failed tickers so far (217): BF.A, BF.B, BFO, BFR, BFRA, BFY, BGCP, BGG, BGIO, BGNE ...

✅ Processed 800/5402 | With data: 515 | Missing: 285
🚫 Failed tickers so far (285): BSTI, BT, BTN, BURG, BVSN, BVXV, BWL.A, BXE, BXG, BXS ...

✅ Processed 1000/5402 | With data: 639 | Missing: 361
🚫 Failed tickers so far (361): CGA, CGIX, CHAP, CHFC, CHFS, CHK, CHKE, CHL, CHMA, CHRA ...

✅ Processed 1200/5402 | With data: 770 | Missing: 430
🚫 Failed tickers so far (430): CPLG, CPLP, CPSI, CPTA, CRAY, CRCM, CRD.A, CRD.B, CREE, CRHM ...

✅ Processed 1400/5402 | With data: 895 | Missing: 505
🚫 Failed

In [14]:
# Filter cleaned_df to keep only tickers that have valid data
valid_tickers = [t for t in cleaned_df["ticker"].unique() if t not in failed_tickers]

print(f"Before filtering: {len(cleaned_df)} rows")
cleaned_df = cleaned_df[cleaned_df["ticker"].isin(valid_tickers)].reset_index(drop=True)
print(f"After filtering: {len(cleaned_df)} rows (only tickers with data available)")


Before filtering: 5402 rows
After filtering: 3515 rows (only tickers with data available)


In [15]:
# Show value counts for the 'sector' column in the cleaned dataframe
print(cleaned_df['sector'].value_counts())

sector
Financial Services        887
Healthcare                493
Technology                458
Industrials               422
Consumer Cyclical         394
Basic Materials           208
Real Estate               201
Energy                    158
Consumer Defensive        156
Utilities                  82
Communication Services     56
Name: count, dtype: int64


In [16]:
# Save cleaned data to processed folder
output_path = r"C:\Users\mukta\OneDrive\Desktop\Algo_project\data\processed\cleaned_companies.csv"
cleaned_df.to_csv(output_path, index=False)

print(f"Cleaned dataset saved successfully at:\n{output_path}")

Cleaned dataset saved successfully at:
C:\Users\mukta\OneDrive\Desktop\Algo_project\data\processed\cleaned_companies.csv


Conversion of Market Cap values from strings to float

In [17]:
import numpy as np 
import pandas as pd
import re
import os
# === Step 1: Load CSV ===
file_path = r"C:\Users\mukta\OneDrive\Desktop\Algo_project\data\processed\cleaned_companies.csv"
print(f"📂 Loading file from: {file_path}")

cleaned_df = pd.read_csv(file_path)
print(f"✅ File loaded successfully. Rows: {len(cleaned_df)}, Columns: {list(cleaned_df.columns)}")

# === Step 2: Conversion Function ===
def convert_market_cap(value):
    """
    Convert market cap strings → numeric USD floats.
    Handles suffixes (T, B, M, K) and symbols like $, commas.
    """
    if pd.isna(value):
        return np.nan
    if isinstance(value, (int, float, np.integer, np.floating)):
        return float(value)

    s = str(value).strip().upper().replace(",", "").replace("$", "")
    multiplier = 1
    if s.endswith("T"):
        multiplier = 1e12
        s = s[:-1]
    elif s.endswith("B"):
        multiplier = 1e9
        s = s[:-1]
    elif s.endswith("M"):
        multiplier = 1e6
        s = s[:-1]
    elif s.endswith("K"):
        multiplier = 1e3
        s = s[:-1]

    s = re.sub(r"[^0-9.]", "", s)
    try:
        return float(s) * multiplier if s else np.nan
    except ValueError:
        return np.nan

print("🔄 Converting 'market cap' column to numeric values...")

# === Step 3: Apply conversion and overwrite the column ===
cleaned_df["market cap"] = cleaned_df["market cap"].apply(convert_market_cap)

# Progress checks
print("✅ Conversion completed successfully.")
print("🔍 Checking conversion result sample:")
print(cleaned_df[["company name", "market cap"]].head())

# === Step 4: Save back to same CSV (overwrite) ===
cleaned_df.to_csv(file_path, index=False)
print(f"💾 File overwritten successfully at:\n{os.path.abspath(file_path)}")


📂 Loading file from: C:\Users\mukta\OneDrive\Desktop\Algo_project\data\processed\cleaned_companies.csv
✅ File loaded successfully. Rows: 3515, Columns: ['ticker', 'company name', 'market cap', 'sector']
🔄 Converting 'market cap' column to numeric values...
✅ Conversion completed successfully.
🔍 Checking conversion result sample:
                    company name    market cap
0      Agilent Technologies Inc.  2.421807e+10
1              Alcoa Corporation  5.374967e+09
2   American Airlines Group Inc.  1.694019e+10
3  Atlantic American Corporation  5.655808e+07
4   Applied Optoelectronics Inc.  3.522182e+08
💾 File overwritten successfully at:
C:\Users\mukta\OneDrive\Desktop\Algo_project\data\processed\cleaned_companies.csv


Risk Bucket distribution

In [18]:
import numpy as np

print("\n📊 Preparing dynamic risk buckets from 'market cap'...")

# 1) Make sure market cap is numeric (just in case) and positive for stats
cleaned_df["market cap"] = pd.to_numeric(cleaned_df["market cap"], errors="coerce")
valid_mcaps = cleaned_df.loc[cleaned_df["market cap"] >= 0, "market cap"].dropna()

print(f"✔ Valid positive market caps found: {valid_mcaps.size}")
if valid_mcaps.empty:
    raise ValueError("No valid positive market caps available to compute risk buckets.")

# 2) Compute summary stats and tertile cutoffs
mcap_min = valid_mcaps.min()
mcap_max = valid_mcaps.max()
q33 = valid_mcaps.quantile(0.33)
q66 = valid_mcaps.quantile(0.66)

print(f"📉 Min market cap: {mcap_min:,.0f}")
print(f"📈 Max market cap: {mcap_max:,.0f}")
print(f"🔪 Tertile cutoffs → q33: {q33:,.0f}, q66: {q66:,.0f}")

# 3) Map each row to a risk bucket:
#    High   : [min .. q33)    (small caps)
#    Medium : [q33 .. q66)    (mid caps)
#    Low    : [q66 .. +inf)   (large caps)
#    NaN/<=0 → None
def map_risk_bucket(mcap: float) -> str | None:
    if pd.isna(mcap) or mcap <= 0:
        return None
    if mcap < q33:
        return "High"
    if mcap < q66:
        return "Medium"
    return "Low"

print("🔄 Assigning risk buckets to rows ...")
cleaned_df["risk_bucket"] = cleaned_df["market cap"].apply(map_risk_bucket)

# 4) Quick distribution check
bucket_counts = cleaned_df["risk_bucket"].value_counts(dropna=False).rename_axis("bucket").reset_index(name="count")
print("✅ Risk bucket distribution (including None for invalid):")
print(bucket_counts)


📊 Preparing dynamic risk buckets from 'market cap'...
✔ Valid positive market caps found: 3515
📉 Min market cap: 12,219
📈 Max market cap: 816,323,510,654
🔪 Tertile cutoffs → q33: 392,109,020, q66: 2,733,615,215
🔄 Assigning risk buckets to rows ...
✅ Risk bucket distribution (including None for invalid):
   bucket  count
0     Low   1195
1    High   1160
2  Medium   1160


Return rows that match sector + risk filters

In [19]:
# ========= Step 6 (Interactive, optimized): Filter by Sector & Risk and show top-10 by market cap =========
import pandas as pd
from IPython.display import display

# --- Ensure market cap is numeric ONCE (skipped if already numeric) ---
if not pd.api.types.is_numeric_dtype(cleaned_df["market cap"]):
    cleaned_df["market cap"] = pd.to_numeric(cleaned_df["market cap"], errors="coerce")

# --- Precompute a normalized sector column once for fast matching ---
sector_clean = cleaned_df["sector"].astype(str).str.strip()
sector_cf = sector_clean.str.casefold()
cleaned_df["_sector_cf"] = sector_cf

# Unique sector list (canonical form) and normalizer map
unique_canonical = sorted(sector_clean.unique())
norm_map = {s.casefold(): s for s in unique_canonical}

# 1) Display example sectors as a numbered list (not an array)
print("\n📚 Example sectors:")
example_list = unique_canonical[:12] if len(unique_canonical) > 12 else unique_canonical
for i, s in enumerate(example_list, start=1):
    print(f"  {i}. {s}")
print(f"🧭 Total unique sectors: {len(unique_canonical)}")

# --- Input: Sector (case-insensitive). Allow 'All' ---
while True:
    user_sector = input("\nEnter sector (e.g., Technology) or 'All': ").strip()
    if not user_sector:
        print("⚠️ Please type a sector or 'All'.")
        continue
    if user_sector.lower() == "all":
        sector_choice_cf = None
        sector_choice_print = "All"
        break
    key = user_sector.casefold()
    if key in norm_map:
        sector_choice_cf = key
        sector_choice_print = norm_map[key]
        break
    print("❌ Sector not found. Please pick from the list above or type 'All'.")

# --- Input: Risk bucket ---
valid_risks = {"low", "medium", "high"}
while True:
    user_risk = input("Enter risk (Low / Medium / High): ").strip().casefold()
    if user_risk in valid_risks:
        risk_choice = user_risk.capitalize()
        break
    print("❌ Invalid risk. Choose from Low / Medium / High.")

print(f"\n🎯 Filtering for sector = '{sector_choice_print}' and risk = '{risk_choice}' ...")

# --- Build masks using the precomputed normalized sector column ---
risk_mask = cleaned_df["risk_bucket"].eq(risk_choice)
if sector_choice_cf is None:
    sector_mask = cleaned_df["_sector_cf"].notna()
else:
    sector_mask = cleaned_df["_sector_cf"].eq(sector_choice_cf)

# --- Filter, sort by market cap desc, take top 10 ---
filtered = cleaned_df.loc[sector_mask & risk_mask].dropna(subset=["market cap"])
filtered = filtered.sort_values("market cap", ascending=False)
top10 = filtered.head(10)

print(f"✅ Matches found: {len(filtered)} (showing top 10 by market cap)\n")

# --- Pretty table: show all columns, with thousands separators for market cap ---
if top10.empty:
    print("⚠️ No companies found for this combination.")
else:
    fmt = {"market cap": "{:,.0f}"} if "market cap" in top10.columns else {}
    styled = (
        top10.style
        .format(fmt)
        .hide(axis="index")
        .set_table_styles([
            {"selector": "th", "props": [("text-align", "left"), ("padding", "6px 10px")]},
            {"selector": "td", "props": [("padding", "6px 10px")]},
            {"selector": "table", "props": [("border-collapse", "collapse")]}
        ])
    )
    print("📋 Top 10 by Market Cap:\n")
    display(styled)



📚 Example sectors:
  1. Basic Materials
  2. Communication Services
  3. Consumer Cyclical
  4. Consumer Defensive
  5. Energy
  6. Financial Services
  7. Healthcare
  8. Industrials
  9. Real Estate
  10. Technology
  11. Utilities
🧭 Total unique sectors: 11

🎯 Filtering for sector = 'Technology' and risk = 'Medium' ...
✅ Matches found: 140 (showing top 10 by market cap)

📋 Top 10 by Market Cap:



ticker,company name,market cap,sector,risk_bucket,_sector_cf
OMCL,Omnicell Inc.,"2,694,979,970",Technology,Medium,technology
QTWO,Q2 Holdings Inc.,"2,687,077,161",Technology,Medium,technology
SAIL,SailPoint Technologies Holdings Inc.,"2,673,599,133",Technology,Medium,technology
BL,BlackLine Inc.,"2,673,230,318",Technology,Medium,technology
NOVT,Novanta Inc.,"2,581,233,698",Technology,Medium,technology
VIAV,Viavi Solutions Inc.,"2,535,487,930",Technology,Medium,technology
IRBT,iRobot Corporation,"2,510,080,664",Technology,Medium,technology
DSGX,The Descartes Systems Group Inc.,"2,464,287,604",Technology,Medium,technology
EB,Eventbrite Inc. Class A,"2,461,837,776",Technology,Medium,technology
GLOB,Globant S.A.,"2,456,637,000",Technology,Medium,technology


Fetching OHLCV data from yfinance for top-10 companies found

In [20]:
top10

,ticker,company name,market cap,sector,risk_bucket,_sector_cf
2382,OMCL,Omnicell Inc.,2.694980e+09,Technology,Medium,technology
2639,QTWO,Q2 Holdings Inc.,2.687077e+09,Technology,Medium,technology
2777,SAIL,SailPoint Technologies Holdings Inc.,2.673599e+09,Technology,Medium,technology
419,BL,BlackLine Inc.,2.673230e+09,Technology,Medium,technology
2276,NOVT,Novanta Inc.,2.581234e+09,Technology,Medium,technology
3310,VIAV,Viavi Solutions Inc.,2.535488e+09,Technology,Medium,technology
1680,IRBT,iRobot Corporation,2.510081e+09,Technology,Medium,technology
943,DSGX,The Descartes Systems Group Inc.,2.464288e+09,Technology,Medium,technology
977,EB,Eventbrite Inc. Class A,2.461838e+09,Technology,Medium,technology
1377,GLOB,Globant S.A.,2.456637e+09,Technology,Medium,technology


In [21]:
# ========= Step 8: Fetch OHLCV for top-10 in one call and PRINT each DF =========
import os
from datetime import datetime
import pandas as pd
import yfinance as yf

# --- Config
PERIOD   = "60d"
INTERVAL = "1d"

# --- Paths based on your repo tree (kept for consistency, but not writing files anymore)
BASE     = r"C:\Users\mukta\OneDrive\Desktop\Algo_project"
PROC_DIR = os.path.join(BASE, "data", "processed", "ohlcv")
os.makedirs(PROC_DIR, exist_ok=True)

# --- Ticker list from your top10 frame
tickers = top10["ticker"].dropna().astype(str).unique().tolist()
if not tickers:
    raise ValueError("No tickers found in top10.")

print(f"\n📥 Downloading OHLCV for {len(tickers)} tickers in one call...")
data = yf.download(
    tickers=" ".join(tickers),
    period=PERIOD,
    interval=INTERVAL,
    group_by="ticker",
    auto_adjust=False,
    progress=False,
    threads=True,
)

if data is None or data.empty:
    raise RuntimeError("Empty response from yfinance.")

# Determine which tickers actually returned data
if isinstance(data.columns, pd.MultiIndex):
    returned = [t for t in tickers if t in set(data.columns.get_level_values(0))]
else:
    # Single-ticker fallback (rare for a 10-ticker request)
    returned = tickers[:1]

missing = sorted(set(tickers) - set(returned))
if missing:
    print(f"⚠️ Missing/failed tickers ({len(missing)}): {missing}")
else:
    print("✅ All requested tickers returned by Yahoo.")

# --- PRINT each ticker's DataFrame (no dict, no Excel) ---
total_rows = 0
for tkr in returned:
    # Extract per-ticker frame
    if isinstance(data.columns, pd.MultiIndex):
        df = data[tkr].copy()
    else:
        # single-ticker fallback
        df = data.copy()

    # Normalize columns and annotate
    df.columns = [c.title() for c in df.columns]  # Open, High, Low, Close, Adj Close, Volume
    df.insert(0, "Ticker", tkr)
    df.index.name = "Date"

    # Convert index to readable dates (YYYY-MM-DD) if datetime-like
    if isinstance(df.index, pd.DatetimeIndex):
        df = df.reset_index()
        df["Date"] = df["Date"].dt.date
        df = df.set_index("Date")

    # Drop empty rows (if any)
    df = df.dropna(how="all")

    # Print header + summary + full DataFrame
    print("\n" + "=" * 80)
    print(f"📊 {tkr} — {df.shape[0]} rows  |  Range: {df.index.min()} → {df.index.max()}")
    print("=" * 80)
    print(df)  # <-- prints ALL rows for this ticker, as requested

    total_rows += df.shape[0]

print("\n" + "-" * 80)
print(f"✅ Finished printing {len(returned)} tickers  |  Total rows printed: {total_rows}")
if missing:
    print(f"📝 Skipped (no data returned): {missing}")
print("-" * 80)



📥 Downloading OHLCV for 10 tickers in one call...
✅ All requested tickers returned by Yahoo.

📊 OMCL — 60 rows  |  Range: 2025-08-01 → 2025-10-24
           Ticker       Open       High        Low      Close  Adj Close  \
Date                                                                       
2025-08-01   OMCL  30.719999  30.889999  29.510000  29.760000  29.760000   
2025-08-04   OMCL  30.070000  31.200001  30.020000  30.420000  30.420000   
2025-08-05   OMCL  30.350000  30.660000  30.070000  30.500000  30.500000   
2025-08-06   OMCL  30.360001  31.280001  30.320000  31.090000  31.090000   
2025-08-07   OMCL  31.230000  31.639999  30.730000  31.030001  31.030001   
2025-08-08   OMCL  31.000000  31.000000  30.320000  30.830000  30.830000   
2025-08-11   OMCL  30.629999  31.410000  30.629999  30.830000  30.830000   
2025-08-12   OMCL  31.090000  31.879999  30.930000  31.540001  31.540001   
2025-08-13   OMCL  31.559999  31.799999  30.780001  31.340000  31.340000   
2025-08-14   OMCL